# This script contains the code to calculate formation energies and save the necessary data for binary oxides.

In [ ]:
import pickle

import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict

from pymatgen.ext.matproj import MPRester
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Element
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter

from matminer.featurizers.site import CrystalNNFingerprint
from matminer.featurizers.structure import SiteStatsFingerprint

# Initialize the MP Rester
mpr = MPRester('pn8XdbGhMrv90STu')


In [ ]:
ele2gs = pickle.load(open("ele2gs.p", "rb"))
ele2gs["O"] = -4.95

In [ ]:
def pourbaix_data_unitary_element(pbx, entries, metal1, ph, voltage):
    """
    Arguments:
    pbx:           the PourbaixDiagram inputs, passed to the function as 
                   entries = mpr.get_pourbaix_entries([metal])
                   pbx_dat = PourbaixDiagram(entries, filter_solids=True)
                   pbx_dat would then be passed in as the argument.
    entries:       A list of Pourbaix Entries, a pymatgen data type of each materials project datapoint considered in 
                   the Pourbaix diagram
    metal1:        One of the metals to consider, passed as a string ie. "Mn"
    ph:            A single value or range of values
    voltage:       A single value or range of values (must match ph in shape)
    metal2:        Other metal to consider, passed as a string ie. "Mn". By default, we set metal2=None, so that we can
                   decide when we are using the function whether we want to use one or two metals.

    
    Returns:
    material_names:     List of names of each material which are considered stable by the function, as a string.
    materials_proj_ids: List of MP ids of each material which are considered stable by the function, as a string.
    energies:           List of ΔG_pbx of each material thats considered stable by the function, as floating point.
    """

    materials_proj_ids = []
    material_names = []
    energies = []
    stable_one = []
    for entry in entries:
        if "ion" not in entry.entry_id and "O" in entry.name and "H" not in entry.name:
            energy = pbx.get_decomposition_energy(entry, pH=ph, V=voltage)
            materials_proj_ids.append(entry.entry_id)
            material_names.append(entry.name)
            energies.append(energy[0])
        
    return material_names, materials_proj_ids, energies


def calc_formation_ene(entry):
    """
    Need to calculate formation energies ourselves because materials project formation energies are inconsistent.
    See https://matsci.org/t/formation-energy-calculation/41574 for further information.
    
    """    
    composition = {str(key):value for (key,value) in entry.composition.items()}
    total_atoms = sum(composition.values())
    
    total_energy = entry.energy_per_atom
    formation_energy = total_energy
    for element in composition:
        formation_energy-=ele2gs[element]*composition[element]/total_atoms
    print(formation_energy)
    print(entry.entry_id)
    return formation_energy


In [ ]:
#unary_oxide_data = defaultdict()
# Want to fill in this with metals of interest
unary_oxide_data = pickle.load(open("unary_oxide_data_self_calc.p", "rb"))

elements = ["Li", "Be", "Na", "Mg", "K", "Ca", "Rb", "Sr", "Cs", "Ba", # Alkalis
            "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",
            "Ga", "Ge", "As", "Se", "Br",
            "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd",
            "In", "Sn", "Sb", "Te", "I",
            "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", 
            "Tl", "Pb", "Bi",
            "La", "Ce", "Nd", "Pr", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu"]

elements = [ 'Si']
elements = ['Al']
# Could vary this range, ie. perhaps we want to look at pH up to 2/3
min_ph = 0
max_ph = 0
min_v = 1
max_v = 2

ssf = SiteStatsFingerprint(
    CrystalNNFingerprint.from_preset('ops', distance_cutoffs=None, x_diff_weight=0),
    stats=('mean', 'std_dev', 'minimum', 'maximum'))


for ele in elements:
    if ele in unary_oxide_data and ele!='Si':
        continue

    # pass the pH and V as an numpy 'meshgrid'.
    pH, V = np.mgrid[
    min_ph : max_ph : 1 * 1j,
    min_v : max_v : 11 * 1j,
    ]
    # here, we get all entries in the materials project database with metal/O/H
    entries = mpr.get_pourbaix_entries([ele])
    # Then, we retrieve a PourbaixDiagram object.
    pbx_dat = PourbaixDiagram(entries, filter_solids=True)
    
    # Then, we use the function in the above cell where we only pass one metal.
    material_names, material_ids, energies = pourbaix_data_unitary_element(pbx_dat, entries, ele,
                                                                      pH, V)
    assert len(material_names)==len(material_ids)==len(energies)
    structures = []
    fingerprints = []
    material_names_ = []
    material_ids_ = []
    energies_ = []
    form_enes_ = []
    print(material_ids)
    for idx, mp_id in enumerate(material_ids):
        try:
            if material_names[idx]=='SiO2(s)' and material_ids[idx]!='mp-546794':
                continue
            structure = mpr.get_structure_by_material_id(mp_id)
            #print(mp_id)
            # exception happens below
            fingerprint = np.array(ssf.featurize(structure))
            # exception happens above
            structures.append(structure)
            fingerprints.append(fingerprint)
            material_names_.append(material_names[idx])
            material_ids_.append(material_ids[idx])
            energies_.append(energies[idx])
            ref_query = mpr.get_entry_by_material_id(material_ids[idx])#, property_data=['formation_energy_per_atom'])
            ref_form_ene = calc_formation_ene(ref_query)
            form_enes_.append(ref_form_ene)
        except ValueError as e :
            # get an error for a Ge oxide
            print(e)
    print('doing commit')
    assert len(fingerprints)==len(structures)==len(material_ids_)
    unary_oxide_data[ele] = {
        "names": material_names_,
        "mp_ids": material_ids_,
        "energies": energies_,
        "structures": structures,
        "fingerprints": fingerprints,
        "formation_energies": form_enes_
    }
    pickle.dump(unary_oxide_data, open("unary_oxide_data_self_calc.p", "wb"))
    

In [ ]:
"""
these saved  nested dictionaries are structured like with an element key, and a dictionrary mapping
properties to list of values for oxides of that given element. 
For each element, maps to the following:
'names': list of material names
'mp_ids': list of materialsproject ids
'energies': list of pbx energies at pH=0, V=[1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]
'structures': list of pymatgen structure files, will be used for oxidation state tracking.
'fingerprints': list of pymatgen featuratization used in materialsproject, will be used to match
simialr materials when trying to predict stability of multimetal materials.
'formation_energies': list of fomration energies for each material.

"""

pickle.dump(unary_oxide_data, open("unary_oxide_data_self_calc.p", "wb"))
